In [3]:
import anndata
import numpy as np
import pandas as pd
import patsy
import random
from scipy.stats import rankdata
from scipy.linalg import svd

/opt/conda/lib/python3.7/site-packages/anndata/_core/anndata.py:21: FutureWarning: pandas.core.index is deprecated and will be removed in a future version.  The public classes are available in the top-level namespace.
  from pandas.core.index import RangeIndex


In [4]:
pheno = pd.read_csv("./data/bladderPheno.csv", index_col=0)
adata = anndata.read_csv("./data/bladderExp.csv")
model_matrix = patsy.dmatrix('cancer', data=pheno)

In [5]:
def calc_res(X, H):
    return X - (H @ X.T).T

def calc_dstat(res, idx):
    u, s, v = svd(res)
    return s[:idx]**2/np.sum(s[:idx]**2)

def num_sv(adata, model_matrix, method='be', vfilter=None, B=20, seed=None):
    if seed is not None:
        try:
            np.random.seed(seed)
        except:
            raise ValueError("seed should be integer")
    
    if vfilter is not None:
        if (vfilter < 100) or (vfilter > adata.shape[0]):
            raise ValueError("The number of genes used in the analysis must be between 100 and", str(adata.shape[0]),"\n")
        adata.obs['base_var'] = np.var(adata.X, ddof=1, axis=1)
        adata = adata[rankdata(-adata.obs['base_var']) < vfilter]
        
    if method not in ['be', 'leek']:
        raise ValueError('method should be "be" or "leek"')
        
    
    if method == 'be':
        H = model_matrix @ np.linalg.inv(model_matrix.T @ model_matrix) @ model_matrix.T
        res = calc_res(adata.X, H)
        idx = int(min(adata.shape) - np.ceil(np.sum(np.diag(H))))
        dstat = calc_dstat(res, idx)
        dstat0 = np.zeros((20, idx))
        for i in range(B):
            tmp_res = np.apply_along_axis(lambda x: random.sample(list(x), len(x)), 1, res)
            tmp_res = calc_res(tmp_res, H)
            dstat0[i, ] = calc_dstat(tmp_res, idx)
            
        psv = np.ones(adata.shape[1])
        for i in range(idx):
            psv[i] = np.mean(dstat0[:, i] >= dstat[i])
            
        for i in range(1, idx):
            psv[i] = max(psv[i-1], psv[i])
            
        nsv = np.sum(psv <= 0.10)
        return nsv

In [6]:
nsv = num_sv(adata, model_matrix, vfilter=2000)

In [7]:
nsv

8

In [35]:
tmp_dstat = dstat0.copy()

In [38]:
for i in range(20):
    tmp_res = np.apply_along_axis(lambda x: random.sample(list(x), len(x)), 1, res)
    tmp_res = calc_res(tmp_res, H)
    tmp_dstat[i, ] = calc_dstat(tmp_res, idx)

In [43]:
len(tmp_dstat[:,0])

20

In [17]:
nsv[0]

array([-1.52588837, -1.18330758, -0.96021931, -0.87189261, -0.85076993,
       -0.73378634, -0.65914147, -0.62621379, -0.62368577, -0.61848538,
       -0.58259559, -0.5785207 , -0.36448924, -0.32489007, -0.31861107,
       -0.31009476, -0.25602786, -0.25379634, -0.23392289, -0.2288119 ,
       -0.1787719 , -0.17114607, -0.15600102, -0.14352124, -0.1434068 ,
       -0.11326098, -0.03575413, -0.0349292 ,  0.04774673,  0.10745056,
        0.18474491,  0.21121891,  0.22181041,  0.23358377,  0.23740871,
        0.2474776 ,  0.24936588,  0.27281125,  0.34551628,  0.34579952,
        0.37355812,  0.38398273,  0.39097316,  0.45512683,  0.47091205,
        0.54661186,  0.55822189,  0.5989886 ,  0.63609989,  0.64489873,
        0.6613028 ,  0.68040855,  0.69275577,  0.74107393,  0.75333905,
        0.89374263,  0.89501119])

In [129]:
get_res(adata, H)

array([[ 0.75333905, -0.73378634, -0.58259559, ...,  0.23358377,
         0.27281125, -0.2288119 ],
       [ 0.14462644, -0.136944  , -0.08742589, ...,  0.0067304 ,
        -0.02547614,  0.01901229],
       [ 0.01899678,  0.33459717,  0.13686424, ..., -0.33102836,
        -0.13553561,  0.17181264],
       ...,
       [-0.12130988,  0.03667486,  0.05075681, ..., -0.08360836,
        -0.00330755,  0.02341512],
       [-0.14431509,  0.12811866,  0.1036934 , ..., -0.01161385,
        -0.04846358,  0.21685457],
       [-0.13430241,  0.1011605 , -0.08471683, ..., -0.07547728,
        -0.0867788 ,  0.09468802]])

In [77]:
res = adata.X - (H @ adata.X.T).T

In [78]:
res

array([[ 0.75333905, -0.73378634, -0.58259559, ...,  0.23358377,
         0.27281125, -0.2288119 ],
       [ 0.14462644, -0.136944  , -0.08742589, ...,  0.0067304 ,
        -0.02547614,  0.01901229],
       [ 0.01899678,  0.33459717,  0.13686424, ..., -0.33102836,
        -0.13553561,  0.17181264],
       ...,
       [-0.12130988,  0.03667486,  0.05075681, ..., -0.08360836,
        -0.00330755,  0.02341512],
       [-0.14431509,  0.12811866,  0.1036934 , ..., -0.01161385,
        -0.04846358,  0.21685457],
       [-0.13430241,  0.1011605 , -0.08471683, ..., -0.07547728,
        -0.0867788 ,  0.09468802]])

In [60]:
res = adata.X - (H @ adata.X.T).T

In [61]:
res

array([[ 0.75333905, -0.73378634, -0.58259559, ...,  0.23358377,
         0.27281125, -0.2288119 ],
       [ 0.14462644, -0.136944  , -0.08742589, ...,  0.0067304 ,
        -0.02547614,  0.01901229],
       [ 0.01899678,  0.33459717,  0.13686424, ..., -0.33102836,
        -0.13553561,  0.17181264],
       ...,
       [-0.12130988,  0.03667486,  0.05075681, ..., -0.08360836,
        -0.00330755,  0.02341512],
       [-0.14431509,  0.12811866,  0.1036934 , ..., -0.01161385,
        -0.04846358,  0.21685457],
       [-0.13430241,  0.1011605 , -0.08471683, ..., -0.07547728,
        -0.0867788 ,  0.09468802]])

In [29]:
U, s, V = svd(res)

In [34]:
res

ArrayView([[-0.19248867, -0.24244738, -0.14333725, ..., -0.15144947,
             0.46047041, -0.22618654],
           [ 0.51133645, -0.45525539,  0.04840577, ...,  1.00150273,
            -0.54247024, -0.60379055],
           [ 0.92283618, -0.93512642, -0.56364453, ...,  0.48257224,
            -0.3997434 , -1.96657499],
           ...,
           [ 0.66303396,  1.4061408 , -1.00597477, ...,  0.0276964 ,
            -0.83719752,  0.40430811],
           [ 0.01596892,  1.00199997, -0.47375429, ...,  0.17953142,
            -0.94904868, -0.0979983 ],
           [ 0.00665659,  0.58328766,  0.05175489, ..., -0.39224699,
            -0.1520212 ,  0.2653291 ]])

In [6]:
vfilter = 2000

In [7]:
adata.obs['base_var'] = np.var(adata.X, ddof=1, axis=1)

In [8]:
rankdata(-adata.obs['base_var']) < vfilter

array([False, False, False, ..., False, False, False])

In [9]:
model_matrix = patsy.dmatrix('cancer', data=pheno)

In [10]:
H = np.dot(np.dot(model_matrix, np.linalg.inv(np.dot(model_matrix.T, model_matrix))), model_matrix.T)

In [11]:
H = model_matrix @ np.linalg.inv(model_matrix.T @ model_matrix) @ model_matrix.T

In [12]:
res = exp - (H @ exp.T).T

In [13]:
res = adata.X - (H @ adata.X.T).T

In [14]:
U, s, V = svd(res)

In [15]:
ndf = int(min(adata.shape) - np.ceil(np.sum(np.diag(H))))

In [37]:
dstat = s[0:ndf]**2/(np.sum(s[0:ndf]**2))

In [17]:
dstat0 = np.zeros((20, ndf))

In [18]:
np.apply_along_axis(lambda x: np.random.choice(x, len(x)), 0, res).T.shape

(57, 22283)

In [19]:
B = 20

In [63]:
for i in range(20):
    tmp_res = np.apply_along_axis(lambda x: np.random.choice(x, len(x)), 0, res)
    tmp_res = tmp_res - (H @ tmp_res.T).T
    tmp_u, tmp_s, tmp_v = svd(tmp_res)
    dstat0[i, ] = tmp_s[:ndf]**2/np.sum(tmp_s[:ndf]**2)

KeyboardInterrupt: 

In [145]:
tmp_res = np.apply_along_axis(lambda x: np.random.choice(x, len(x)), 0, res).T

In [148]:
H @ tmp_res

array([[ 0.17559573, -0.20564208, -0.11653037, ...,  0.23290481,
         0.15061016,  0.01089744],
       [ 0.17559573, -0.20564208, -0.11653037, ...,  0.23290481,
         0.15061016,  0.01089744],
       [ 0.17559573, -0.20564208, -0.11653037, ...,  0.23290481,
         0.15061016,  0.01089744],
       ...,
       [ 0.17038651,  0.15664643,  0.20443351, ...,  0.04624177,
        -0.15531808,  0.21290659],
       [ 0.17038651,  0.15664643,  0.20443351, ...,  0.04624177,
        -0.15531808,  0.21290659],
       [ 0.17038651,  0.15664643,  0.20443351, ...,  0.04624177,
        -0.15531808,  0.21290659]])

In [25]:
dstat0.shape

(20, 54)

In [31]:
psv = np.ones(adata.shape[1])

In [40]:
for i in range(ndf):
    psv[i] = np.mean(dstat0[1] >= dstat[1])

In [41]:
psv

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 1., 1., 1.])

In [42]:
for i in range(1, ndf):
    psv[i] = max(psv[i-1], psv[i])

In [43]:
psv

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 1., 1., 1.])

In [46]:
nsv = np.sum(psv <= 0.10)

In [47]:
nsv

54

In [56]:
dstat0[7,]

array([0.03626053, 0.03310931, 0.0320006 , 0.03182664, 0.03118746,
       0.02922345, 0.02896368, 0.02754427, 0.02701594, 0.02583733,
       0.02498315, 0.02463308, 0.02445544, 0.02343785, 0.02298894,
       0.02223634, 0.0216695 , 0.02124326, 0.02077855, 0.02049972,
       0.02027807, 0.02002122, 0.01904444, 0.01884891, 0.01869137,
       0.01863098, 0.01774699, 0.01745962, 0.0174366 , 0.01671175,
       0.01612549, 0.01567593, 0.01550268, 0.01488633, 0.01482996,
       0.01376904, 0.01309854, 0.01288056, 0.01284336, 0.01268508,
       0.01257718, 0.0123005 , 0.01228625, 0.01195046, 0.01102823,
       0.01095455, 0.01057984, 0.01031806, 0.01027596, 0.00976455,
       0.00925173, 0.00813488, 0.00682093, 0.00669488])

In [53]:
dstat[7]

0.024160863787624033

In [65]:
tmp_res = np.apply_along_axis(lambda x: np.random.choice(x, len(x)), 1, res)

In [66]:
tmp_res

array([[-0.31009476,  0.37355812, -0.65914147, ..., -0.85076993,
        -0.0349292 ,  0.10745056],
       [-0.06225154,  0.07452347, -0.49606798, ...,  0.05857899,
        -0.14743803, -0.09119604],
       [-0.61116346,  0.28203784,  0.60916106, ...,  0.13043991,
        -0.09409031, -0.12471994],
       ...,
       [ 0.05636992, -0.08360836, -0.05590997, ..., -0.06188045,
         0.09795513, -0.12934933],
       [ 0.0128746 , -0.00428471,  0.12811866, ...,  0.02152323,
         0.22249317, -0.13585424],
       [-0.12598531, -0.12598531, -0.09418265, ..., -0.10237853,
        -0.09854047, -0.10237853]])

In [67]:
tmp_res.shape

(22283, 57)

In [5]:
x = np.random.randn(20, 5)

In [6]:
x

array([[ 1.73673761,  1.89791391, -2.10677342, -0.14891209,  0.58306155],
       [-2.25923303,  0.13723954, -0.70121322, -0.62078008, -0.47961976],
       [ 1.20973877, -1.07518386,  0.80691921, -0.29078347, -0.22094764],
       [-0.16915604,  1.10083444,  0.08251052, -0.00437558, -1.72255825],
       [ 1.05755642, -2.51791281, -1.91064012,  1.30645535,  0.42439764],
       [-0.46916808, -1.00607253, -0.92192304,  0.62640453,  0.3571767 ],
       [-0.01871855,  0.22117127,  0.69860411,  0.83451002, -1.66563784],
       [-2.90143984, -0.04863853, -0.04537833,  1.47533391, -0.51369344],
       [-0.66844251, -1.03332741,  2.5052391 ,  0.10662364,  0.36308511],
       [ 1.11232645, -0.59172085,  1.5448004 ,  2.47639208, -2.87470857],
       [ 2.3960241 ,  0.13124583,  0.30439887,  1.17660585,  0.12989416],
       [ 0.21369857,  0.09996282, -0.04785691,  0.37494644,  0.40462887],
       [ 0.33815844,  0.19862082, -1.39696286, -0.74150159,  1.16431059],
       [-0.91628325, -0.2530479 ,  1.6

In [7]:
import random
np.apply_along_axis(lambda x: random.sample(list(x), len(x)), 1, x)

array([[ 1.89791391, -0.14891209, -2.10677342,  1.73673761,  0.58306155],
       [-0.47961976,  0.13723954, -0.70121322, -0.62078008, -2.25923303],
       [ 0.80691921, -0.22094764, -1.07518386,  1.20973877, -0.29078347],
       [-0.16915604, -1.72255825,  1.10083444,  0.08251052, -0.00437558],
       [-2.51791281,  1.30645535, -1.91064012,  1.05755642,  0.42439764],
       [-0.46916808,  0.62640453,  0.3571767 , -0.92192304, -1.00607253],
       [ 0.22117127,  0.83451002,  0.69860411, -0.01871855, -1.66563784],
       [-0.04537833, -0.04863853, -0.51369344,  1.47533391, -2.90143984],
       [ 0.10662364, -0.66844251,  0.36308511,  2.5052391 , -1.03332741],
       [ 2.47639208,  1.5448004 , -2.87470857,  1.11232645, -0.59172085],
       [ 0.12989416,  0.13124583,  1.17660585,  2.3960241 ,  0.30439887],
       [ 0.37494644,  0.40462887,  0.21369857, -0.04785691,  0.09996282],
       [-1.39696286,  0.33815844,  0.19862082,  1.16431059, -0.74150159],
       [-0.91628325,  1.69172841, -0.2

In [23]:
random.sample(list(range(10)))

TypeError: sample() missing 1 required positional argument: 'k'